In [1]:
from awsglue.context import GlueContext
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from pyspark.sql import DataFrame
import boto3
import sys
import json
import re
from awsglue.job import Job

In [2]:
sys.argv +=["--JOB_NAME","glue-script","--landingbucket","gbs-idp-dashboard-landing-dev","--SECURITY_CONFIGURATION","P2pLogisticsGlueSecurityConfiguration","--stagingbucket","gbs-idp-dashboard-staging-dev",'--configfile',"idp-dashboard/etl/conf/landing_to_staging.json","--configbucket","data-analytics-config-dev","--format","parquet"]
args = getResolvedOptions(
        sys.argv, [ "JOB_NAME","configbucket","configfile", "landingbucket", "stagingbucket","format"]
    )
print(args)

{'job_bookmark_option': 'job-bookmark-disable', 'job_bookmark_from': None, 'job_bookmark_to': None, 'JOB_ID': None, 'JOB_RUN_ID': None, 'SECURITY_CONFIGURATION': 'P2pLogisticsGlueSecurityConfiguration', 'encryption_type': None, 'RedshiftTempDir': None, 'TempDir': None, 'JOB_NAME': 'glue-script', 'configbucket': 'data-analytics-config-dev', 'configfile': 'idp-dashboard/etl/conf/landing_to_staging.json', 'landingbucket': 'gbs-idp-dashboard-landing-dev', 'stagingbucket': 'gbs-idp-dashboard-staging-dev', 'format': 'parquet'}


In [5]:
sc = SparkContext()
glueContext = GlueContext(sc)
glueContext._sc._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider")
glueContext._sc._jsc.hadoopConfiguration().set("fs.s3a.access.key",os.environ['AWS_ACCESS_KEY_ID'])
glueContext._sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key",os.environ['AWS_SECRET_ACCESS_KEY'] )
glueContext._sc._jsc.hadoopConfiguration().set("fs.s3a.session.token",os.environ['AWS_SESSION_TOKEN'])
glueContext.spark_session.conf.set("spark.local.dir", "C:\\rpa-logs\\")
job = Job(glueContext)
spark = glueContext.spark_session


KeyError: 'JOB_NAME'

In [3]:
def format_header(name):
    name = re.sub(r'([A-Z](?=[a-z]))', r'_\1', name)
    name = re.sub(r'((?<=[a-z0-9])[A-Z])', r'_\1', name)
    name = name.lower()
    name = re.sub(r'[^\w]', '_', name)
    name = re.sub(r'^(_+)', '', name)
    name = re.sub(r'(_+)$', '', name)
    name = re.sub(r'(_{2,})', '_', name)
    return name
def typecasting(df: DataFrame,castinfo) -> DataFrame:
        for header in df.columns:
            if header in castinfo:
                df = df.withColumn(header, df[header].cast(castinfo[header]))
        return df

def csvtoparquet(spark,sourceinfo,args):
    print(sourceinfo)
    inputDF=spark.read.option("delimiter",sourceinfo['formatOptions']['sep']).option("nullValue", sourceinfo['formatOptions']['nullValue']).option("escape", sourceinfo['formatOptions']['escape']).option("quote", sourceinfo['formatOptions']['quote']).option("header",sourceinfo['formatOptions']['header']) \
                                   .option("inferSchema",sourceinfo['formatOptions']['inferschema']).csv(sourceinfo['landing_prefix'].replace('{landingbucket}',args['landingbucket']))
    inputDF=typecasting(inputDF,sourceinfo['cast'])
    for column in inputDF.columns:
        inputDF = inputDF.withColumnRenamed(column, format_header(column))
    inputDF.repartition(1).write.mode("overwrite").format(args['format']).save(sourceinfo['staging_prefix'].replace('{stagingbucket}',args['stagingbucket']))

In [4]:
if __name__ == "__main__":
    args = getResolvedOptions(
        sys.argv, [ 'JOB_NAME',"configbucket","configfile", "landingbucket", "stagingbucket","format"]
    )
    s3 = boto3.resource("s3")
    config_s3_object = s3.Object(args["configbucket"], args["configfile"])
    config = json.loads(config_s3_object.get()["Body"].read())['Source']
    sc = SparkContext.getOrCreate()
    glueContext = GlueContext(sc)
    job = Job(glueContext)
    job.init(args['JOB_NAME'], args)
    spark = glueContext.spark_session
    for sourceinfo in config:
        csvtoparquet(spark,config[sourceinfo],args)

{'landing_prefix': 's3://{landingbucket}/malaysiaBefore', 'staging_prefix': 's3://{stagingbucket}/malaysia-before', 'cast': {'VendorId': 'int', 'PostalCode': 'int', 'VendorCode': 'int'}, 'formatOptions': {'header': 'true', 'sep': ',', 'quote': '"', 'escape': '"', 'inferschema': 'true', 'nullValue': 'NULL'}, 'partition_count': 1}
{'landing_prefix': 's3://{landingbucket}/malaysiaAfter', 'staging_prefix': 's3://{stagingbucket}/malaysia-after', 'cast': {'VendorId': 'int', 'PostalCode': 'int', 'VendorCode': 'int'}, 'formatOptions': {'header': 'true', 'sep': ',', 'quote': '"', 'escape': '"', 'inferschema': 'true', 'nullValue': 'NULL'}, 'partition_count': 1}


In [11]:
config

{'malaysiaBefore': {'landing_prefix': 's3://{landingbucket}/malaysiaBefore',
  'staging_prefix': 's3://{stagingbucket}/malaysia-before',
  'cast': {'VendorId': 'int', 'PostalCode': 'int', 'VendorCode': 'int'},
  'formatOptions': {'header': 'true',
   'sep': ',',
   'quote': '"',
   'escape': '"',
   'inferschema': 'true',
   'nullValue': 'NULL'},
  'partition_count': 1},
 'malaysiaAfter': {'landing_prefix': 's3://{landingbucket}/malaysiaAfter',
  'staging_prefix': 's3://{stagingbucket}/malaysia-after',
  'cast': {'VendorId': 'int', 'PostalCode': 'int', 'VendorCode': 'int'},
  'formatOptions': {'header': 'true',
   'sep': ',',
   'quote': '"',
   'escape': '"',
   'inferschema': 'true',
   'nullValue': 'NULL'},
  'partition_count': 1}}

In [37]:
malaysiaAfterDF.withColumn("PostalCode",col('PostalCode').cast("int")).withColumn("VendorCode",col('VendorCode').cast("int")).withColumn("VendorId",col('VendorId').cast("int")).where("EmailDate is not null").select('EmailDate').show()

+--------------------+
|           EmailDate|
+--------------------+
|Tue, 18 Aug 2020 ...|
|Tue, 18 Aug 2020 ...|
|Tue, 18 Aug 2020 ...|
|Tue, 18 Aug 2020 ...|
|Wed, 19 Aug 2020 ...|
|Wed, 19 Aug 2020 ...|
|Wed, 19 Aug 2020 ...|
|Wed, 19 Aug 2020 ...|
|Tue, 18 Aug 2020 ...|
|Tue, 18 Aug 2020 ...|
|Tue, 18 Aug 2020 ...|
|Wed, 19 Aug 2020 ...|
|Wed, 19 Aug 2020 ...|
|Wed, 19 Aug 2020 ...|
|Wed, 19 Aug 2020 ...|
|Wed, 19 Aug 2020 ...|
|Wed, 19 Aug 2020 ...|
|Wed, 19 Aug 2020 ...|
|Wed, 19 Aug 2020 ...|
|Wed, 19 Aug 2020 ...|
+--------------------+
only showing top 20 rows

